In [1]:
import os
import pandas as pd
import tweepy
import sqlite3
import gensim
import nltk
import numpy as np
import time
import random
import secrets
import itertools
import yaml
import re
import requests
from bs4 import BeautifulSoup as bs

C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### PARAMETERS

In [2]:
maxday=5   #maksimum hari yang di reply.
person_to_follow = 3    #jumlah orang yg di follow

In [3]:
#AMBIL FILE KONFIGURASI DARI TWITTER.YML
with open('twitter.yml') as f:
    twit = yaml.load(f)
    
consumer_key = twit['consumer_key']
consumer_secret = twit['consumer_secret']
access_token = twit['access_token']
access_token_secret = twit['access_token_secret']

In [4]:
import datetime
def is_today(date):
    ckdt = datetime.datetime.strptime(date,'%Y-%m-%d').date()
    if ckdt == datetime.datetime.today().date():
        return True
    return False

def daydiff(date1,date2):
    d0 = datetime.datetime.strptime(date1,'%Y-%m-%d').date()
    d1 = datetime.datetime.strptime(date2,'%Y-%m-%d').date()
    delta = d1 - d0
    return delta.days

# Configure
def twit_get_username(username,result='result.csv',limit=10,count=10):
    try:
        os.remove(result)
    except Exception as e:
        pass

    c = twint.Config()
    #c.Search = "#maling"
    c.Username= username
    c.Format = ""
    c.Debug = False
    c.Store_csv = True
    # CSV Fieldnames
    #following should be used as fieldnames in a list:
    #id,date,time,timezone,user_id,username,tweet,replies,retweets,likes,hashtags,link
    #c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = result
    c.Limit=limit
    c.Count=count
    # Run
    twint.Search(c)

    df = pd.read_csv(result)
    return df

def user_follower(username,result='result.csv'):
    try:
        os.remove(result)
    except:
        pass

    c = twint.Config()
    #c.Search = "#maling"
    c.Username= username
    c.Format = ""
    c.Debug = False
    c.Store_csv = True
    # CSV Fieldnames
    #following should be used as fieldnames in a list:
    #id,date,time,timezone,user_id,username,tweet,replies,retweets,likes,hashtags,link
    #c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = result
    # Run
    output = twint.run.Followers(c)

    df = pd.read_csv(result)
    return df

def user_following(username,result='result.csv'):
    try:
        os.remove(result)
    except:
        pass

    c = twint.Config()
    #c.Search = "#maling"
    c.Username= username
    c.Format = ""
    c.Debug = False
    c.Store_csv = True
    # CSV Fieldnames
    #following should be used as fieldnames in a list:
    #id,date,time,timezone,user_id,username,tweet,replies,retweets,likes,hashtags,link
    #c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = result
    # Run
    output = twint.run.Following(c)

    df = pd.read_csv(result)
    return df



### Connect DB & Routines

In [5]:
from tweetdb import *

### Routine Logic - tweepy

In [6]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

def twp_update_status(msg):
    try:
        api.update_status(msg)
        print("Update status to:",msg)
    except Exception as e:
        print("error updating status:",e)
        
def twp_reply_post(msg,replyID):
    try:
        rets = api.update_status(msg, in_reply_to_status_id=replyID)
        print("Update status to:",msg)
        return rets
    except Exception as e:
        print("error updating status:",e)
        
def twp_cool_post(msg, username, userfollower):
    res = twit_get_username(username)
    msgid = res.iloc[postnum].id
    #username = res.iloc[postnum]
    follower = user_following(userfollower)
    import random
    follist = follower['GundalaPutraTop'].values.tolist()
    random.shuffle(follist)
    mention = "@"+follist[0]+" @"+follist[1]+" @"+follist[2]
    msge = str("@")+str(username)+" "+str(msg)+" "+str(mention)
    print("Pesan:",msge, "replyto:",msgid)
    api.update_status(msge, msgid)

def twp_follow(UserIDs):
    for userID in UserIDs:
        try:
            api.create_friendship(userID)
            print("Try to follow:", userID)
        except Exception as e:
            print(e)
            continue

# Routine BOT - AI

In [7]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11002]
[nltk_data]     getaddrinfo failed>


False

In [8]:
from chat_wordvec import *

In [9]:
import keras
model = keras.models.load_model("./model/model-ep2-100.mdl")

C:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
def pred(sentence):
    predictions=model.predict(np.array([swv_ar(twl(sentence))]))
    result=decode(predictions)
    print(result)
    return result

def decode(pred):
    result=[]
    for i in range(maxsent):
        res = tex2vec.wv.similar_by_vector(pred[0][i])[0][0]
        result.append(res)
        result = rem_list(result)
        strj = ' '.join(result)
    return strj       

In [11]:
posmsg = [
    "Ini bagus banget, ",
    "Saya suka banget sama ini, ",
    "Wah asik banget ini yah. ",
    "Bagooooooossssss, ",
    "I love this, ",
    "Kapan nih berita gini?, ",
    "I love this post, ",
    "Postingan yang sangat bermutu, ",
    "Kapan?, ",
    "Terus kasih lagi gan..., ",
    "I like this, ",
    "Lanjuuuut gan, ",
    "Komen yang bermutu, ",
    "Twit ini sadis kerennya, ",
    "He he heh e, ",
    "Lanjoot, "
]

def ranmsg():
    a = secrets.choice(posmsg)
    return a

In [12]:
def get_id_follower(username,count=20):
    result=[]
    maxc=0
    try:
        for user in tweepy.Cursor(api.followers, screen_name=username).items(count):
            maxc += 1
            result.append(user.screen_name)
            if maxc > count:
                break
    except:
        print("Error - get follower")
        result=['aagym']
    return result

In [13]:
def get_rnd(rlist,count=3):
    result = []
    for i in range(count):
        a1 = random.randint(0,len(rlist)-1)
        result.append(rlist[a1])
    return result

In [14]:

def replyer(m):
    m_user = m.user.screen_name
    print("Username:",m_user)
    msg_id = m.id
    m_text = m.text
    if not db_is_replied(msg_id):

        flwr = get_id_follower(str(m_user))
        reslt=get_rnd(flwr)
        
        #db_insert_follower(m_user,reslt)
        #twp_follow(reslt)
        #print("Success adding follower of:", m_user)
        recap=cleanhtml(m_text)+","
        sender="@"+m_user+" "
        msg1 = ranmsg()
        msg2 = pred(m_text)
        msg3 = kaskus_search(m_text)

        reslt = secrets.choice(reslt)
        
        tagthem = ' @'+' @'.join(reslt)
        fullmsg = recap+msg1+msg2+","+msg3+" "    #+tagthem
        
        fullmsg = nltk.word_tokenize(fullmsg)
        fullmsg = rem_list(fullmsg)
        fullmsg = rem_at(fullmsg)
        fullmsg = join_url(fullmsg)
        fullmsg = " ".join(fullmsg)
        
        print("\n\nOUTPUT: ",fullmsg, '\n=======================\n\n')
        
        if realpost:
            rets = twp_reply_post(fullmsg,None)
        
            #twp_cool_post(fullmsg,m_user,m_user)
            if rets:
                db_replied_this_id(msg_id,str(rets.id))
                db_replied_this_id(str(rets.id),str(rets.id))
            else:
                print("Error posting message =============")
    else:
        print("already replied")
        pass

### RUTIN KASKUS SEARCH

In [15]:
from kaskus_comment import *

# TWEEPY INITIALIZE

In [16]:
if __name__ == '__main__':
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth)

    #m = public_tweets[0]
    #replyer(m)

#cool_post("Keren gan wow-mantap","irzaip", "iriantomo")

#contoh sebuah reply
#api.update_status("@iriantomo Status ini bro kerens.", in_reply_to_status_id=1001139203487318016)

In [17]:
    #print(m.user.screen_name)
    #print(m.user.followers_count)

# BALAS SEMUA TWEET DI HOMEPAGE

In [18]:
import time
import random

In [ ]:
    #Bener bener di posting
    realpost = True

    public_tweets = api.home_timeline()

    for tweet in public_tweets:
        print(tweet.text)
        replyer(tweet)
        #db_is_replied(tweet.id)
        time.sleep(random.randint(100,350))

Good night Singapore 😊❤ @ Singapore https://t.co/YRbWD9MfGa
Username: MerryRiana


C:\Anaconda3\envs\tensorflow\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Lupa Lupa Lupa semen sepertinya sasaran Ludwig sarapan tawarikh melanggar


OUTPUT:  Good night Singapore 😊❤ Singapore https://t.co/YRbWD9MfGa , Kapan ? , Lupa Lupa Lupa semen sepertinya sasaran Ludwig sarapan tawarikh melanggar , Ane senin tanggal 10 gan bermgkat spore . 


Update status to: Good night Singapore 😊❤ Singapore https://t.co/YRbWD9MfGa , Kapan ? , Lupa Lupa Lupa semen sepertinya sasaran Ludwig sarapan tawarikh melanggar , Ane senin tanggal 10 gan bermgkat spore .
RT @fontool: قراءة بالمقام السوداني جميله جداً
ما أعظم كتاب الله ❤️ https://t.co/D9tItfoc1W
Username: Agus_Iman
Sentence overflow: ['rt', 'fontool', 'قراءة', 'بالمقام', 'السوداني', 'جميله', 'ما', 'أعظم', 'كتاب', 'الله', 'https']
Problem at phrase: fontool
Problem at phrase: قراءة
Problem at phrase: بالمقام
Problem at phrase: السوداني
Problem at phrase: جميله
Problem at phrase: ما
Problem at phrase: أعظم
Problem at phrase: كتاب
Lupa Lupa Lupa semen sepertinya sepertinya sepertinya sepertinya sepertinya sepertinya


## Kalau mau balas post orang

In [21]:
def reply_post_user(username, postnum=0):
    posts=api.user_timeline(id=username)
    replyer(posts[postnum])

# PSEUDO CODE

id = ambil_id_msg()
check_msg_di_db()
liat_user_msg()
list_follower_usermsg()
ambil_3_random_follower()
ambil_random_jawaban()
ambil_prediksi_twit_user()
buat_msg()
post_msg()
catat_posting_di_db()
follow_3_user()

# SANDBOX

# CLEAN UP  - Don't Forget

In [22]:
#db.close()

In [43]:
from data import *